<a href="https://colab.research.google.com/github/kmswin1/Pytorch-Study/blob/master/Week2_ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
accelerator = 'cu80' if os.path.exists('/opt/bin/nvidia-smi') else 'cpu'
!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision

tcmalloc: large alloc 1073750016 bytes == 0x5c0f2000 @  0x7fe281c832a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641


In [2]:
# 본인의 구글 드라이브와 연동하는 과정 1
# 실행 후 밑에 나오는 링크에 들어가서 나오는 코드를 밑에 입력
# Downloaded content "Sample upload file content" 라는 메시지가 나오면 OK

# (런타임을 재시작한 경우) 아무 메시지 없이 계속 기다리는 상황이 있는데, 밑에 생기는 네모 빈 칸에 엔터 입력

!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

uploaded = drive.CreateFile({'title': 'Sample upload.txt'})
uploaded.SetContentString('Sample upload file content')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

downloaded = drive.CreateFile({'id': uploaded.get('id')})
print('Downloaded content "{}"'.format(downloaded.GetContentString()))

Uploaded file with ID 1L4KqLWX4VCsHpa4E2Gvfwq6XWDLNU5Ln
Downloaded content "Sample upload file content"


In [3]:
# 본인의 구글 드라이브와 연동하는 과정 2
# 실행 후 밑에 나오는 링크에 들어가서 나오는 코드를 밑에 입력
# Please enter the verification code: Access token retrieved correctly. 라는 메시지가 나오면 OK

# (런타임을 재시작한 경우) 아무 메시지 없이 계속 기다리는 상황이 있는데, 밑에 생기는 네모 빈 칸에 엔터 입력
# fuse: mountpoint is not empty
# fuse: if you are sure this is safe, use the 'nonempty' mount option
# 라는 메시지가 나오면 그냥 무시

# 또한 필요한 라이브러리를 설치
# 좀 시간이 걸림

!apt-get update > /dev/null
!apt-get install -y -qq software-properties-common  module-init-tools g++ > /dev/null
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass

!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

!mkdir -p drive
!google-drive-ocamlfuse drive

W: Failed to fetch http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu/dists/bionic/InRelease  Could not connect to ppa.launchpad.net:80 (91.189.95.83), connection timed out
W: Some index files failed to download. They have been ignored, or old ones used instead.
W: Failed to fetch http://ppa.launchpad.net/alessandro-strada/ppa/ubuntu/dists/bionic/InRelease  Could not connect to ppa.launchpad.net:80 (91.189.95.83), connection timed out
W: Failed to fetch http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu/dists/bionic/InRelease  Unable to connect to ppa.launchpad.net:http:
W: Some index files failed to download. They have been ignored, or old ones used instead.


KeyboardInterrupt: ignored

In [5]:
import os
import sys
import timeit
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import scipy as sp
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

%matplotlib inline

#plt.rc('text', usetex = True)
mpl.rcParams["font.family"] = "serif"
mpl.rcParams["font.size"] = "15"

print('python version : ', sys.version)
print('numpy version : ', np.version.version)
print('scipy version : ', sp.__version__)
print('matplotlib version :', mpl.__version__)
print('pytorch version : ', torch.__version__)
print('Cuda : ', torch.cuda.is_available())

python version :  3.6.7 (default, Oct 22 2018, 11:32:17) 
[GCC 8.2.0]
numpy version :  1.14.6
scipy version :  1.1.0
matplotlib version : 3.0.2
pytorch version :  0.4.0
Cuda :  True


In [0]:
!ls ./drive/2019_pytorch_study

In [0]:
batchSize = 256
learningRate = 0.01
momentum = 0.9
decay = 5 * 10e-4
epochNum = 80
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [0]:
# original code
# https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/02-intermediate/deep_residual_network/main.py

def conv3x3(inCh, outCh, stride=1):
    return nn.Conv2d(inCh, outCh, kernel_size=3, 
                     stride=stride, padding=1, bias=False)

class ResBlock(nn.Module):

  def __init__(self, inCh, outCh, stride = 1, downsample = None):

    super(ResBlock, self).__init__() # 부모 클래스의 생성자를 1번만 호출되도록 하는 문법

    layers = list()

    layers.append(conv3x3(inCh, outCh, stride))
    layers.append(nn.BatchNorm2d(outCh))
    layers.append(nn.ReLU(inplace = True))
    layers.append(conv3x3(outCh, outCh))
    layers.append(nn.BatchNorm2d(outCh))

    # nn.ModuleList has no forward() -> 네트워크가 연결이 안됌
    #self.main = nn.ModuleList(layers)
    # nn.Sequential has a forward() -> 네트워크가 연결이 됌
    self.main = nn.Sequential(*layers) # layers 의 원소를 하나하나 넣어주는 것  
    # def func(list) list = [a,b,c,d] 일때,  func(*list) -> func(a,b,c,d) 펼쳐줌 dict = {'a':1, 'b':2, 'c':2} 일때,  func(**dict) -> func(a = 1, b = 3, c = 4) 

    self.relu = nn.ReLU(inplace = True)
    self.downsample = downsample

  def forward(self, x):

    out = self.main(x)
    
    if self.downsample:
      
      x = self.downsample(x)
    
    return self.relu(x + out)

class ResNet(nn.Module):
    
  def __init__(self, block, layers, num_classes=10):
        
    super(ResNet, self).__init__()
    
    self.inCh = 16
    self.conv = conv3x3(3, 16)
    self.bn = nn.BatchNorm2d(16)
    self.relu = nn.ReLU(inplace=True)

    self.layer1 = self._makeLayer(block, 16, layers[0])
    self.layer2 = self._makeLayer(block, 32, layers[1], 2)
    self.layer3 = self._makeLayer(block, 64, layers[2], 2)
    self.pool = nn.AvgPool2d(8)
    self.fc = nn.Linear(64, num_classes)
        
  def _makeLayer(self, block, outCh, blocks, stride = 1): # python3 부터는 _ 두개 쓰면 외부에서 호출이 안됌. e.g) __fname
    
    downsample = None
    
    if (stride != 1) or (self.inCh != outCh):

      downsample = nn.Sequential(
        conv3x3(self.inCh, outCh, stride=stride),
        nn.BatchNorm2d(outCh)
        )

    layers = list()

    layers.append(block(self.inCh, outCh, stride, downsample))
    self.inCh = outCh
    
    for i in range(1, blocks):

      layers.append(block(outCh, outCh))
    
    return nn.Sequential(*layers)
    
  def forward(self, x):
    
    out = self.conv(x)
    out = self.bn(out)
    out = self.relu(out)
    out = self.layer1(out)
    out = self.layer2(out)
    out = self.layer3(out)
    out = self.pool(out)
    out = out.view(out.size(0), -1)
    out = self.fc(out)
    
    return out
    
model = ResNet(ResBlock, [2, 2, 2]).to(device)

In [8]:
testData = torch.randn(1, 3, 32, 32).to(device) # 모델 테스팅
print(testData)
print(testData.shape)

tensor([[[[ 0.2346, -0.5857, -0.0730,  ..., -0.9872, -0.9511,  1.2328],
          [-0.0092,  0.5327, -0.2060,  ...,  0.2366,  0.6440, -1.7629],
          [-0.7285, -1.2310, -0.8111,  ..., -0.5005,  0.0931, -0.0525],
          ...,
          [-1.3158,  0.2682, -0.1332,  ..., -1.2353, -0.6965, -1.1995],
          [ 0.7493,  0.0866,  0.1833,  ..., -0.7906,  0.1386, -0.6817],
          [-0.7547,  0.5595, -0.5206,  ..., -1.5933, -1.1643,  0.6670]],

         [[-0.6093,  1.2670, -1.1926,  ...,  0.0862,  3.6485,  1.2364],
          [ 0.3695, -0.9166,  0.5998,  ...,  0.1962,  1.1341, -1.0397],
          [-0.5808,  0.2837, -2.4869,  ...,  0.6642, -0.6618,  1.2045],
          ...,
          [-1.2729,  2.3247, -3.3395,  ...,  0.2623, -0.5492, -0.0801],
          [-1.5856, -0.1025,  1.9844,  ..., -1.6170,  0.5047, -0.9888],
          [ 0.3709, -1.0465, -0.0181,  ..., -0.4128, -0.4770,  0.5440]],

         [[-1.3045,  1.3287, -0.6260,  ...,  0.7975,  2.1231, -1.0344],
          [ 1.4535,  0.5622, -

In [11]:
result = model(testData)
print(result)

tensor([[ 0.2421, -0.1623, -0.1870,  0.3535, -0.5501,  0.0450,  0.4461,
          0.1761,  0.5154, -0.2440]], device='cuda:0')


In [16]:
a = nn.Softmax(dim = 1)
b = torch.randn(2, 4)
print(b)
print(a(b))
print(torch.max(a(b), dim = 1))

tensor([[-0.4854,  1.4386,  0.2828,  0.7860],
        [-0.1329, -0.2803, -0.4005,  0.0374]])
tensor([[ 0.0737,  0.5047,  0.1589,  0.2628],
        [ 0.2622,  0.2263,  0.2006,  0.3109]])
(tensor([ 0.5047,  0.3109]), tensor([ 1,  3]))
